In [1]:
!pip install -q folium
!pip install -q transformers
!pip install -q datasets
!pip install -q huggingface_hubz

     |████████████████████████████████| 4.2 MB 5.0 MB/s 
     |████████████████████████████████| 596 kB 81.9 MB/s 
     |████████████████████████████████| 86 kB 6.3 MB/s 
     |████████████████████████████████| 6.6 MB 78.5 MB/s 
     |████████████████████████████████| 346 kB 5.1 MB/s 
     |████████████████████████████████| 212 kB 59.9 MB/s 
     |████████████████████████████████| 140 kB 1.8 MB/s 
     |████████████████████████████████| 86 kB 6.9 MB/s 
     |████████████████████████████████| 1.1 MB 82.1 MB/s 
     |████████████████████████████████| 127 kB 75.0 MB/s 
     |████████████████████████████████| 271 kB 94.8 MB/s 
     |████████████████████████████████| 94 kB 4.1 MB/s 
     |████████████████████████████████| 144 kB 86.0 MB/s 
     |████████████████████████████████| 112 kB 94.2 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 req

In [2]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
import datasets
import pandas as pd
from huggingface_hub import notebook_login

In [3]:
from sklearn.metrics import accuracy_score, f1_score

In [4]:
notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [5]:
num_labels = 7

In [6]:
model_ckpt = 'klue/roberta-large'
model = AutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels=num_labels,)
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

Downloading:   0%|          | 0.00/547 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.decoder.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.out_proj.weight', 'classifier.out_proj.bias', 'cl

Downloading:   0%|          | 0.00/375 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/243k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/734k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/173 [00:00<?, ?B/s]

In [7]:
data = datasets.load_dataset('csv', data_files='/content/train_data.csv')['train']
data = data.train_test_split(test_size=0.2, seed=100)
data = data.remove_columns('index')

Using custom data configuration default-794d29d82e8bac8e


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-794d29d82e8bac8e/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [8]:
data

DatasetDict({
    train: Dataset({
        features: ['title', 'topic_idx'],
        num_rows: 36523
    })
    test: Dataset({
        features: ['title', 'topic_idx'],
        num_rows: 9131
    })
})

In [9]:
def tokenize(batch):
    return tokenizer(batch["title"], padding=True, truncation=True)

In [10]:
encoded_data = data.map(tokenize, batched=True, batch_size=None)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [11]:
encoded_data

DatasetDict({
    train: Dataset({
        features: ['title', 'topic_idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 36523
    })
    test: Dataset({
        features: ['title', 'topic_idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9131
    })
})

In [12]:
encoded_data = encoded_data.rename_column('topic_idx', 'label')

In [13]:
encoded_data

DatasetDict({
    train: Dataset({
        features: ['title', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 36523
    })
    test: Dataset({
        features: ['title', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9131
    })
})

In [14]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

In [15]:
batch_size = 64
logging_steps = len(encoded_data['train']) // batch_size
model_name = f"{model_ckpt}-finetuned-klue-roberta-tc"
training_args = TrainingArguments(output_dir=model_name,
                                    num_train_epochs=10,
                                    learning_rate=2e-5,
                                    per_device_train_batch_size=batch_size,
                                    per_device_eval_batch_size=batch_size,
                                    weight_decay=0.01,
                                    evaluation_strategy="epoch",
                                    disable_tqdm=False,
                                    logging_steps=logging_steps,
                                    push_to_hub=True,
                                    log_level="error")

In [ ]:
trainer = Trainer(model=model, args=training_args,
                    compute_metrics=compute_metrics,
                    train_dataset=encoded_data['train'],
                    eval_dataset=encoded_data['test'],
                    tokenizer=tokenizer)
trainer.train()

Cloning https://huggingface.co/june2003/roberta-large-finetuned-klue-roberta-tc into local empty directory.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.426100,0.343547,0.885993,0.883926
2,0.267500,0.341533,0.890045,0.889179
3,0.185000,0.407067,0.885336,0.884150


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.426100,0.343547,0.885993,0.883926
2,0.267500,0.341533,0.890045,0.889179
3,0.185000,0.407067,0.885336,0.884150


In [ ]:
data_test = datasets.load_dataset('csv', data_files='/content/test_data.csv')['train']

In [ ]:
data_test = data_test.remove_columns('index')

In [ ]:
encoded_test = data_test.map(tokenize, batched=True, batch_size=None)

In [ ]:
encoded_test

In [ ]:
preds_output = trainer.predict(encoded_data['test'])

In [ ]:
preds_output.metrics

In [24]:
import pickle

In [25]:
with open('klue_roberta-large.pkl','wb') as f:
    pickle.dump(preds_output.metrics,f)